In [25]:
from operator import itemgetter				#this functionality is NOT needed. It may help slightly, but you can definitely ignore it completely.

#DO NOT CHANGE!
def read_train_file():
	'''
	HELPER function: reads the training files containing the words and corresponding tags.
	Output: A tuple containing 'words' and 'tags'
	'words': This is a nested list - a list of list of words. See it as a list of sentences, with each sentence itself being a list of its words.
	For example - [['A','boy','is','running'],['Pick','the','red','cube'],['One','ring','to','rule','them','all']]
	'tags': A nested list similar to above, just the corresponding tags instead of words. 
	'''						
	f = open('train','r')
	words = []
	tags = []
	lw = []
	lt = []
	for line in f:
		s = line.rstrip('\n')
		w,t= s.split('/')[0],s.split('/')[1]
		if w=='###':
			words.append(lw)
			tags.append(lt)
			lw=[]
			lt=[]
		else:
			lw.append(w)
			lt.append(t)
	words = words[1:]
	tags = tags[1:]
	assert len(words) == len(tags)
	f.close()
	return (words,tags)








#NEEDS TO BE FILLED!
def train_func(train_list_words, train_list_tags):
    
    '''
	This creates dictionaries storing the transition and emission probabilities - required for running Viterbi. 
	INPUT: The nested list of words and corresponding nested list of tags from the TRAINING set. This passing of correct lists and calling the function
	has been done for you. You only need to write the code for filling in the below dictionaries. (created with bigram-HMM in mind)
	OUTPUT: The two dictionaries

	HINT: Keep in mind the boundary case of the starting POS tag. You may have to choose (and stick with) some starting POS tag to compute bigram probabilities
	for the first actual POS tag.
    '''

    dict2_tag_follow_tag = {}    
    
    '''Nested dictionary to store the transition probabilities
    each tag X is a key of the outer dictionary with an inner dictionary as the corresponding value
    The inner dictionary's key is the tag Y following X
    and the corresponding value is the number of times Y follows X - convert this count to probabilities finally before returning 
    for example - { X: {Y:0.33, Z:0.25}, A: {B:0.443, W:0.5, E:0.01}} (and so on) where X,Y,Z,A,B,W,E are all POS tags
    so the first key-dictionary pair can be interpreted as "there is a probability of 0.33 that tag Y follows tag X, and 0.25 probability that Z follows X"
    '''
    dict2_word_tag = {}
    """Nested dictionary to store the emission probabilities.
	Each word W is a key of the outer dictionary with an inner dictionary as the corresponding value
	The inner dictionary's key is the tag X of the word W
	and the corresponding value is the number of times X is a tag of W - convert this count to probabilities finally before returning
	for example - { He: {A:0.33, N:0.15}, worked: {B:0.225, A:0.5}, hard: {A:0.1333, W:0.345, E:0.25}} (and so on) where A,N,B,W,E are all POS tags
	so the first key-dictionary pair can be interpreted as "there is a probability of 0.33 that A is the POS tag for He, and 0.15 probability that N is the POS tag for He"
	"""


	#      *** WRITE YOUR CODE HERE ***    
        
    l = 0       
    for line in train_list_words:
        w = 0
        for word in line:
            tag = train_list_tags[l][w]
            if word in dict2_word_tag:
                if tag in dict2_word_tag[word]:
                    dict2_word_tag[word][tag] += 1
                else:
                    dict2_word_tag[word][tag] = 1
            else:
                dict2_word_tag[word] = {}
                if tag in dict2_word_tag[word]:
                    dict2_word_tag[word][tag] += 1
                else:
                    dict2_word_tag[word][tag] = 1
            w += 1
        l += 1
    
    for k,v in dict2_word_tag.items():
        count = 0
        temp = k
        for k1,v1 in v.items():
            count += v1
        if count <= 5:
            dict2_word_tag.pop(k,None)
            dict2_word_tag[temp] = {'<UNK>':1.0}
        for k1,v1 in v.items():
            v1 /= count
            dict2_word_tag[k][k1] = v1
	  
    for line in train_list_tags:
        t = 0
        for tag in line:
            t += 1
            if t == 1:
                prev = 'START'
            if tag in dict2_tag_follow_tag:
                if prev in dict2_tag_follow_tag[tag]:
                    dict2_tag_follow_tag[tag][prev] += 1
                else:
                    dict2_tag_follow_tag[tag][prev] = 1
            else:
                dict2_tag_follow_tag[tag] = {}
                if prev in dict2_tag_follow_tag[tag]:
                    dict2_tag_follow_tag[tag][prev] += 1
                else:
                    dict2_tag_follow_tag[tag][prev] = 1
            prev = tag
    
    for k,v in dict2_tag_follow_tag.items():
        count = 0
        for k1,v1 in v.items():
            count += v1
        for k1,v1 in v.items():
            v1 /= count
            dict2_tag_follow_tag[k][k1] = v1











	# END OF YOUR CODE	

    return (dict2_tag_follow_tag, dict2_word_tag)



#NEEDS TO BE FILLED!
def assign_POS_tags(test_words, dict2_tag_follow_tag, dict2_word_tag):

    '''
	This is where you write the actual code for Viterbi algorithm. 
	INPUT: test_words - this is a nested list of words for the TEST set
	       dict2_tag_follow_tag - the transition probabilities (bigram), filled in by YOUR code in the train_func
	       dict2_word_tag - the emission probabilities (bigram), filled in by YOUR code in the train_func
	OUTPUT: a nested list of predicted tags corresponding to the input list test_words. This is the 'output_test_tags' list created below, and returned after your code
	ends.

	HINT: Keep in mind the boundary case of the starting POS tag. You will have to use the tag you created in the previous function here, to get the
	transition probabilities for the first tag of sentence...
	HINT: You need not apply sophisticated smoothing techniques for this particular assignment.
	If you cannot find a word in the test set with probabilities in the training set, simply tag it as 'N'. 
	So if you are unable to generate a tag for some word due to unavailibity of probabilities from the training set,
	just predict 'N' for that word.

    '''



    output_test_tags = []    #list of list of predicted tags, corresponding to the list of list of words in Test set (test_words input to this function)


    #      *** WRITE YOUR CODE HERE *** 
    
    for line in test_words:
        tag_list = []
        v = [[0 for i in range(len(dict2_tag_follow_tag.keys()))] for j in range(len(line)+1)]
        
        for i in range(len(dict2_tag_follow_tag.keys())):
            v[0][i] = 1
        
        i = 1
        for word in line:
            w = word
            if word not in dict2_word_tag.keys():
                tag_list.append('N')
                continue
            k = 0
            for tag in dict2_tag_follow_tag.keys():
                j = 0
                max_prod = 0
                if i == 1:
                    a = v[i-1][j]
                    if 'START' not in dict2_tag_follow_tag[tag]:
                        b = 0.5
                    else:
                        b = dict2_tag_follow_tag[tag]['START']
                    if tag not in dict2_word_tag[w] or w not in dict2_word_tag:
                        c = 0
                    else:
                        c = dict2_word_tag[w][tag]
                    max_prod = a*b*c
                else:
                    for prev_tag in dict2_tag_follow_tag.keys():
                        a = v[i-1][j]
                        if prev_tag not in dict2_tag_follow_tag[tag]:
                            b = 0.5
                        else:
                            b = dict2_tag_follow_tag[tag][prev_tag]
                        if tag not in dict2_word_tag[w] or w not in dict2_word_tag:
                            c = 0
                        else:
                            c = dict2_word_tag[w][tag]
                        max_prod = max(max_prod,a*b*c)
                        j += 1
                v[i][k] = max_prod
                k += 1
            temp = 0
            best_ind = 0
            for l in range(len(dict2_tag_follow_tag.keys())):
                if v[i][l] > temp:
                    temp = v[i][l]
                    best_ind = l
            tag_list.append(list(dict2_tag_follow_tag.keys())[best_ind])
            i += 1
        
        output_test_tags.append(tag_list)
                
















	# END OF YOUR CODE

    return output_test_tags









# DO NOT CHANGE!
def public_test(predicted_tags):
	'''
	HELPER function: Takes in the nested list of predicted tags on test set (prodcuced by the assign_POS_tags function above)
	and computes accuracy on the public test set. Note that this accuracy is just for you to gauge the correctness of your code.
	Actual performance will be judged on the full test set by the TAs, using the output file generated when your code runs successfully.
	'''

	f = open('test_public_labeled','r')
	words = []
	tags = []
	lw = []
	lt = []
	for line in f:
		s = line.rstrip('\n')
		w,t= s.split('/')[0],s.split('/')[1]
		if w=='###':
			words.append(lw)
			tags.append(lt)
			lw=[]
			lt=[]
		else:
			lw.append(w)
			lt.append(t)
	words = words[1:]
	tags = tags[1:]
	assert len(words) == len(tags)
	f.close()
	public_predictions = predicted_tags[:len(tags)]
	assert len(public_predictions)==len(tags)

	correct = 0
	total = 0
	flattened_actual_tags = []
	flattened_pred_tags = []
	for i in range(len(tags)):
		x = tags[i]
		y = public_predictions[i]
		if len(x)!=len(y):
			print(i)
			print(x)
			print(y)
			break
		flattened_actual_tags+=x
		flattened_pred_tags+=y
	assert len(flattened_actual_tags)==len(flattened_pred_tags)
	correct = 0.0
	for i in range(len(flattened_pred_tags)):
		if flattened_pred_tags[i]==flattened_actual_tags[i]:
			correct+=1.0
	print('Accuracy on the Public set = '+str(correct/len(flattened_pred_tags)))



# DO NOT CHANGE!
if __name__ == "__main__":
	words_list_train = read_train_file()[0]
	tags_list_train = read_train_file()[1]

	dict2_tag_tag = train_func(words_list_train,tags_list_train)[0]
	dict2_word_tag = train_func(words_list_train,tags_list_train)[1]

	f = open('test_full_unlabeled','r')

	words = []
	l=[]
	for line in f:
		w = line.rstrip('\n')
		if w=='###':
			words.append(l)
			l=[]
		else:
			l.append(w)
	f.close()
	words = words[1:]
	test_tags = assign_POS_tags(words, dict2_tag_tag, dict2_word_tag)
	assert len(words)==len(test_tags)

	public_test(test_tags)

	#create output file with all tag predictions on the full test set

	f = open('output','w')
	f.write('###/###\n')
	for i in range(len(words)):
		sent = words[i]
		pred_tags = test_tags[i]
		for j in range(len(sent)):
			word = sent[j]
			pred_tag = pred_tags[j]
			f.write(word+'/'+pred_tag)
			f.write('\n')
		f.write('###/###\n')
	f.close()

	print('OUTPUT file has been created')


Accuracy on the Public set = 0.9361914257228315
OUTPUT file has been created


In [15]:
dict2_tag_tag

{'#': {'#': 0.13636363636363635,
  'D': 0.13636363636363635,
  'I': 0.36363636363636365,
  'P': 0.045454545454545456,
  'R': 0.09090909090909091,
  'T': 0.09090909090909091,
  'V': 0.13636363636363635},
 '$': {',': 0.009489916963226572,
  '-': 0.03914590747330961,
  ':': 0.013048635824436536,
  'C': 0.0771055753262159,
  'D': 0.09134045077105575,
  'I': 0.3867141162514828,
  'J': 0.018979833926453145,
  'N': 0.00830367734282325,
  'P': 0.01542111506524318,
  'R': 0.06287069988137604,
  'T': 0.15895610913404506,
  'V': 0.11743772241992882,
  'W': 0.0011862396204033216},
 "'": {"'": 0.009873060648801129,
  ',': 0.4245416078984485,
  '.': 0.3215796897038082,
  'F': 0.005641748942172073,
  'I': 0.0014104372355430183,
  'J': 0.039492242595204514,
  'N': 0.1692524682651622,
  'P': 0.0014104372355430183,
  'R': 0.005641748942172073,
  'V': 0.021156558533145273},
 ',': {',': 0.00019837333862328903,
  '-': 0.006347946835945249,
  'C': 0.07895258877206904,
  'D': 0.0029756000793493354,
  'E': 0.

In [16]:
dict2_word_tag

{'When': {'W': 1.0},
 'such': {'J': 0.8854166666666666, 'P': 0.11458333333333333},
 'claims': {'N': 0.7777777777777778, 'V': 0.2222222222222222},
 'and': {'C': 1.0},
 'litigation': {'N': 1.0},
 'extend': {'V': 1.0},
 'beyond': {'I': 0.8571428571428571, 'R': 0.14285714285714285},
 'the': {'D': 0.9994926433282597,
  'J': 0.0002536783358701167,
  'N': 0.0002536783358701167},
 'period': {'N': 1.0},
 ',': {',': 1.0},
 'syndicates': {'N': 1.0},
 'can': {'M': 0.9866666666666667, 'N': 0.013333333333333334},
 'their': {'P': 1.0},
 'accounting': {'N': 0.4, 'V': 0.6},
 '.': {'.': 1.0},
 'Lloyd': {'N': 1.0},
 "'s": {'P': 0.8606719367588933, 'V': 0.13932806324110672},
 'said': {'V': 1.0},
 'there': {'E': 0.8307692307692308, 'R': 0.16923076923076924},
 'are': {'V': 1.0},
 'currently': {'R': 1.0},
 '115': {'C': 1.0},
 'open': {'J': 0.6428571428571429,
  'N': 0.07142857142857142,
  'V': 0.2857142857142857},
 'account': {'N': 0.8125, 'V': 0.1875},
 'years': {'N': 1.0},
 'involving': {'V': 1.0},
 '68': 